<a href="https://colab.research.google.com/github/Amarjeet4242/repo1/blob/main/marketing_advitismeant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marketing  Advertisement

 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


 Load Cleaned Datasets

In [ ]:

from google.colab import files
import pandas as pd

print("Please upload: transactions_clean.csv")
uploaded = files.upload()
transactions = pd.read_csv(list(uploaded.keys())[0])

print("Please upload: ads_clean.csv")
uploaded = files.upload()
ads = pd.read_csv(list(uploaded.keys())[0])

print("Please upload: abtest_clean.csv")
uploaded = files.upload()
abtest = pd.read_csv(list(uploaded.keys())[0])

print("Uploaded successfully!")
transactions.head(), ads.head(), abtest.head()


Prepare Transactions Data

In [ ]:

transactions['order_date'] = pd.to_datetime(
    transactions['order_date'],
    dayfirst=True,
    errors='coerce'
)

transactions['customer_id'] = transactions['customer_id'].astype(str)
transactions['order_id'] = transactions['order_id'].astype(str)
transactions['revenue'] = pd.to_numeric(transactions['revenue'], errors='coerce').fillna(0)


Step 4 — RFM Calculation

In [ ]:
snapshot = transactions['order_date'].max() + pd.Timedelta(days=1)

rfm = transactions.groupby('customer_id').agg(
    Recency=('order_date', lambda x: (snapshot - x.max()).days),
    Frequency=('order_id', 'count'),
    Monetary=('revenue', 'sum')
).reset_index()

rfm['AOV'] = rfm['Monetary'] / rfm['Frequency']


CLV Calculation

CLV = (AOV × Frequency) × Customer Lifespan:

In [ ]:
rfm['CLV'] = rfm['AOV'] * rfm['Frequency'] * 1
rfm.head()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

 Ad KPIs (Marketing Metrics)

In [ ]:
ads['CTR'] = ads['clicks'] / ads['impressions']
ads['CPC'] = ads['spend'] / ads['clicks']
ads['CPA'] = ads['spend'] / ads['conversions']
ads['ROAS'] = ads['revenue'] / ads['spend']

ads.head()


ML Model (Predict Revenue)

In [ ]:
X = rfm[['Recency','Frequency','Monetary','AOV']]
y = rfm['Monetary']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

preds = model.predict(X_test)

mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

mae, r2


A/B Testing (Two-Proportion Z-test)

Simple Plots
Top 20 CLV Customers

In [ ]:
top = rfm.sort_values('CLV', ascending=False).head(20)
plt.figure(figsize=(10,5))
plt.bar(top['customer_id'].astype(str), top['CLV'])
plt.xticks(rotation=90)
plt.title('Top 20 CLV Customers')
plt.show()


Spend vs Revenue

In [ ]:
plt.scatter(ads['spend'], ads['revenue'])
plt.xlabel('Spend')
plt.ylabel('Revenue')
plt.title('Ad Spend vs Revenue')
plt.grid(True)
plt.show()


Step 10 — Save Output Files

In [ ]:
rfm.to_csv('rfm_output.csv', index=False)
ads.to_csv('ads_kpi_output.csv', index=False)
abtest.to_csv('abtest_output.csv', index=False)
